Course Instructor: Bernd Neumayr, JKU

# UE05: SHACL

Complete the 10 tasks (1 point per task) in the 4. SHACL sheet of SemAI.jar first and then transfer them to this notebook.

For each task include:

 A headline including the task number

 The task description

 The data graph and your solution (the shapes graph) in executable form

 After executing the validation, print out the validation results in tabular form.



**Preparations**

In [13]:
# Install required packages
!pip install -q rdflib 
!pip3 install -q pyshacl    

**Imports and Functions**

In [14]:
# Imports
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace, Dataset
from rdflib.namespace import FOAF , XSD , RDFS 
from rdflib.plugins.sparql.processor import SPARQLResult
from rdflib.namespace import NamespaceManager

from pyshacl import validate

import pandas as pd

def sparql_select(graph,query,use_prefixes=True):
  results = graph.query(query)          # execute the query against the graph, resulting in a rdflib.plugins.sparql.processor.SPARQLResult
  rows = [ { var : res[var].n3(graph.namespace_manager) if (isinstance(res[var],URIRef) and use_prefixes) else res[var] for var in results.vars } for res in results ]     
                                        # construct a list of dictionaries, as intermediate format to construct the pandas DataFrame, use prefixes to abbreviate URIs                
  return pd.DataFrame(rows,columns=results.vars)        
                                        # return a pandas DataFrame constructed from the list of dictionaries, with the variables from the result set as columns      

def validation_report_as_dataframe(validation_report):
  df = sparql_select(results_graph,"""
		SELECT  ?focusNode ?resultPath ?value ?sourceConstraintComponent ?sourceShape ?resultMessage
		WHERE
  		{ ?vr	a sh:ValidationResult ;
						sh:focusNode ?focusNode ;
						sh:sourceConstraintComponent ?sourceConstraintComponent ;
						sh:sourceShape ?sourceShape ;
						sh:resultMessage ?resultMessage .					 
				OPTIONAL { ?vr sh:value ?value . }
				OPTIONAL { ?vr sh:resultPath ?resultPath . }
  		}
  """,use_prefixes=True)
  return df

def shacl_validate(dg,sg):
  return validate(dg,shacl_graph=sg,
      inference='rdfs',
      abort_on_first=False,
      allow_infos=False,
      allow_warnings=False,
      meta_shacl=False,
      advanced=False,
      js=False,
      debug=False)  
  

def shacl_validate_with_rules(dg,sg):
	return validate(dg,shacl_graph=sg,
      inference='rdfs',
      abort_on_first=False,
      allow_infos=False,
      allow_warnings=False,
      meta_shacl=False,
      advanced=True,
      iterate_rules=True, inplace=True,
      js=False,
      debug=False)

1. TASK

Wenn eine Person eine x:knows Beziehung hat, dann sollte das Ziel dieser Beziehung als IRI angegeben sein.

In [15]:

dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>
<Peter>  rdf:type  <Person> ;
        :knows    <John> ;
        :knows    [ :name  "Peter" ] .

<John>  :knows  [ :name   "Mary" ] .
""")
sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

[] a sh:NodeShape;
sh:targetClass <Person>;
sh:property[sh:path :knows; sh:nodeKind sh:IRI].

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/properties/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Peter> ;
            sh:resultMessage "Value is not of Node Kind sh:IRI" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NodeKindConstraintComponent ;
            sh:sourceShape [ sh:nodeKind sh:IRI ;
                    sh:path :knows ] ;
            sh:value [ a rdfs:Resource ;
                    :name "Peter" ] ] .




,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/Peter>,:knows,n0486eec8113b4e6698e27d1091207055b1,sh:NodeKindConstraintComponent,nbe834b463b8e45259b2d8fd1e664436ab2,Value is not of Node Kind sh:IRI


2. TASK

Die Klassen Man und Woman sind disjunkt. Hinweis: verwenden Sie sh:not.

In [16]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>
<YoungMan>  rdfs:subClassOf  <Man> .

<John>  rdf:type  <YoungWoman> , <YoungMan> .

<Mary>  rdf:type  <Woman> .

<Peter>  rdf:type  <Man> .

<YoungWoman>  rdfs:subClassOf  <Woman> .

<Jane>  rdf:type  <YoungWoman> .

<Susi>  rdf:type  <YoungWoman> , <Man> .
""")
sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<ManShape> a sh:NodeShape;
    sh:targetClass <Man>;
    sh:not [ sh:class <Woman> ].

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)



@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Susi> ;
            sh:resultMessage "Node <http://example.org/entities/Susi> conforms to shape [ sh:class <http://example.org/entities/Woman> ]" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NotConstraintComponent ;
            sh:sourceShape <http://example.org/entities/ManShape> ;
            sh:value <http://example.org/entities/Susi> ],
        [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/John> ;
            sh:resultMessage "Node <http://example.org/entities/John> conforms to shape [ sh:class <http://example.org/entities/Woman> ]" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NotConstraintComponent ;
            sh:sourc

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/John>,None,<http://example.org/entities/John>,sh:NotConstraintComponent,<http://example.org/entities/ManShape>,Node <http://example.org/entities/John> confor...
1,<http://example.org/entities/Susi>,None,<http://example.org/entities/Susi>,sh:NotConstraintComponent,<http://example.org/entities/ManShape>,Node <http://example.org/entities/Susi> confor...


3. TASK

Wenn eine Person eine :knows Beziehung hat, dann muss das Ziel dieser Beziehung eine Person sein.

In [17]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>
<Peter>  rdf:type  <Person> ;
        :knows    <Mary> ;
        :knows    [ :name  "Mary" ] ;
        :knows    [ rdf:type  <Person> ] .

<Jane>  :knows  <Jim> .
""")
sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<PersonKnowsShape> a sh:nodeShape;
sh:targetClass <Person>;
sh:property[sh:path :knows; sh:class <Person>].

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/properties/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Peter> ;
            sh:resultMessage "Value does not have class <http://example.org/entities/Person>" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape _:na4149773a0f64ee780ef9109dc4cc84fb1 ;
            sh:value [ a rdfs:Resource ;
                    :name "Mary" ] ],
        [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Peter> ;
            sh:resultMessage "Value does not have class <http://example.org/entities/Person>" ;
            sh:resultPath :knows ;
            sh:resultSeverit

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/Peter>,:knows,n14625f2746fc4c478f71a2ff45db6002b1,sh:ClassConstraintComponent,na4149773a0f64ee780ef9109dc4cc84fb1,Value does not have class <http://example.org/...
1,<http://example.org/entities/Peter>,:knows,<http://example.org/entities/Mary>,sh:ClassConstraintComponent,na4149773a0f64ee780ef9109dc4cc84fb1,Value does not have class <http://example.org/...


4. TASK

Jede Person hat genau ein Alter (:age). Eine Person ist mindestens 0 und maximal 150 Jahre alt. Erwachsene (Klasse Adult) sind mindestens 19 Jahre alt. Senioren sind mindestens 65 Jahre alt. Kinder sind maximal 12 Jahre alt.
Verwenden Sie sh:minInclusive und sh:maxInclusive

5. TASK

Jede Person hat maximal einen Namen. Der Name muss ein Literal sein und vom Datentyp String. Personen dürfen abgesehen von :name und rdf:type keine weiteren Eigenschaften haben.

In [18]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>
<John>  rdf:type  <MalePerson> ;
        :knows    <Mary> .

<FemalePerson>  rdfs:subClassOf  <Person> .

<Mary>  rdf:type  <FemalePerson> ;
        :name     "Mary" .

<Bibi>  rdf:type  <Person> ;
        :name     "Jim" , "Bibi" .

<Jane>  rdf:type  <Person> ;
        :name     2343 .

<Jim>   rdf:type  <Person> ;
        :name     <Jim> .

<MalePerson>  rdfs:subClassOf  <Person> .
""")
sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<PersonShape> a sh:NodeShape;
sh:targetClass <Person>;
sh:closed true;
sh:ignoredProperties (rdf:type);
sh:property <NameShape>.

<NameShape> a sh:PropertyShape;
sh:path :name;
sh:maxCount 1;
sh:datatype xsd:string;
sh:nodeKind sh:Literal.
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/properties/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Jim> ;
            sh:resultMessage "Value is not Literal with datatype xsd:string" ;
            sh:resultPath :name ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:DatatypeConstraintComponent ;
            sh:sourceShape <http://example.org/entities/NameShape> ;
            sh:value <http://example.org/entities/Jim> ],
        [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Jim> ;
            sh:resultMessage "Value is not of Node Kind sh:Literal" ;
            sh:resultPath :name ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NodeKindConstraintComponent ;
            sh:sourceShape <ht

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/John>,:knows,<http://example.org/entities/Mary>,sh:ClosedConstraintComponent,<http://example.org/entities/PersonShape>,Node <http://example.org/entities/John> is clo...
1,<http://example.org/entities/Jim>,:name,<http://example.org/entities/Jim>,sh:DatatypeConstraintComponent,<http://example.org/entities/NameShape>,Value is not Literal with datatype xsd:string
2,<http://example.org/entities/Jim>,:name,<http://example.org/entities/Jim>,sh:NodeKindConstraintComponent,<http://example.org/entities/NameShape>,Value is not of Node Kind sh:Literal
3,<http://example.org/entities/Jane>,:name,2343,sh:DatatypeConstraintComponent,<http://example.org/entities/NameShape>,Value is not Literal with datatype xsd:string
4,<http://example.org/entities/Bibi>,:name,None,sh:MaxCountConstraintComponent,<http://example.org/entities/NameShape>,More than 1 values on <http://example.org/enti...


6. TASK

Städte sind via Property :inCountry Ländern zugeordnet. Europäische Städte sind europäischen Ländern zugeordnet. Österreichische Städte sind Austria zugeordnet.

In [19]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>
<Paris>  rdf:type   <EuropeanCity> ;
        :inCountry  <France> .

<AustrianCity>  rdfs:subClassOf  <EuropeanCity> .

<Salzburg>  rdf:type  <AustrianCity> ;
        :inCountry  <Germany> .

<EuropeanCountry>  rdfs:subClassOf  <Country> .

<NewYork>  rdf:type  <City> ;
        :inCountry  <USA> .

<Germany>  rdf:type  <EuropeanCountry> .

<EuropeanCity>  rdfs:subClassOf  <City> .

<Vienna>  rdf:type  <AustrianCity> ;
        :inCountry  <Austria> .

<Austria>  rdf:type  <EuropeanCountry> .
""")
sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>
<CityInCountryShape> a sh:NodeShape;
sh:targetClass <City>;
sh:property[sh:path :inCountry; sh:class <Country>].

<EuropeanCityInCountryShape> a sh:NodeShape;
sh:targetClass <EuropeanCity>;
sh:property[sh:path :inCountry; sh:class <EuropeanCountry>].

<AustrianCityInCountryShape> a sh:NodeShape;
sh:targetClass <AustrianCity>;
sh:property[sh:path :inCountry; sh:in (<Austria>)].
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/properties/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Paris> ;
            sh:resultMessage "Value does not have class <http://example.org/entities/EuropeanCountry>" ;
            sh:resultPath :inCountry ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape [ sh:class <http://example.org/entities/EuropeanCountry> ;
                    sh:path :inCountry ] ;
            sh:value <http://example.org/entities/France> ],
        [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/NewYork> ;
            sh:resultMessage "Value does not have class <http://example.org/entities/Country>" 

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/Paris>,:inCountry,<http://example.org/entities/France>,sh:ClassConstraintComponent,nc97547d7672b4cd0a710376f0a390f72b2,Value does not have class <http://example.org/...
1,<http://example.org/entities/NewYork>,:inCountry,<http://example.org/entities/USA>,sh:ClassConstraintComponent,nc97547d7672b4cd0a710376f0a390f72b1,Value does not have class <http://example.org/...
2,<http://example.org/entities/Paris>,:inCountry,<http://example.org/entities/France>,sh:ClassConstraintComponent,nc97547d7672b4cd0a710376f0a390f72b1,Value does not have class <http://example.org/...
3,<http://example.org/entities/Salzburg>,:inCountry,<http://example.org/entities/Germany>,sh:InConstraintComponent,nc97547d7672b4cd0a710376f0a390f72b3,Value <http://example.org/entities/Germany> no...


7. TASK

Die Property :worksFor darf nur Personen (als Subjekt) und Organisation (als Objekt) miteinander verbinden.
Hinweis: Lösen Sie die Aufgabe ohne Property Shapes.

In [20]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>
<Mary>  :worksFor  <JKU> .

<Jane>  rdf:type   <Person> ;
        :worksFor  <LMU> .

<Jim>   rdf:type   <Man> ;
        :worksFor  <LMU> .

<University>  rdfs:subClassOf  <Organization> .

<Bob>   rdf:type   <Person> ;
        :worksFor  [ rdf:type  <Organization> ] .

<JKU>   rdf:type  <University> .
""")
sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>
<WorksForSubjectsShape> a sh:NodeShape;
sh:class <Person>;
sh:targetSubjectsOf :worksFor.

<WorksForTargetShape> a sh:NodeShape;
sh:class <Organization>;
sh:targetObjectsOf :worksFor.
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/LMU> ;
            sh:resultMessage "Value does not have class <http://example.org/entities/Organization>" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape <http://example.org/entities/WorksForTargetShape> ;
            sh:value <http://example.org/entities/LMU> ],
        [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Jim> ;
            sh:resultMessage "Value does not have class <http://example.org/entities/Person>" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape <http://example.org/entities/WorksForSubjectsShape> ;
         

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/LMU>,None,<http://example.org/entities/LMU>,sh:ClassConstraintComponent,<http://example.org/entities/WorksForTargetShape>,Value does not have class <http://example.org/...
1,<http://example.org/entities/Mary>,None,<http://example.org/entities/Mary>,sh:ClassConstraintComponent,<http://example.org/entities/WorksForSubjectsS...,Value does not have class <http://example.org/...
2,<http://example.org/entities/Jim>,None,<http://example.org/entities/Jim>,sh:ClassConstraintComponent,<http://example.org/entities/WorksForSubjectsS...,Value does not have class <http://example.org/...


8. TASK

Peter kennt ausschließlich Männer, die in Österreich leben.

In [21]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<John>  rdf:type  <Man> ;
        :livesIn  <Austria> .

<Mary>  rdf:type  <Woman> ;
        :knows    <Jim> ;
        :livesIn  <Austria> .

<Hans>  rdf:type  <Man> .

<Peter>  :knows  <Josef> , <Hans> , <Franz> , <John> , <Mary> .

<Franz>  rdf:type  <Man> ;
        :livesIn  <Germany> .
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<PeterShape> a sh:NodeShape;
sh:targetNode <Peter>;
sh:property[
sh:path :knows;
sh:node <PeterKnowsShape>].

<PeterKnowsShape> a sh:NodeShape;
sh:class <Man>;
sh:property [sh:path :livesIn; sh:in (<Austria>)].

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/properties/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Peter> ;
            sh:resultMessage "Value does not conform to Shape <http://example.org/entities/PeterKnowsShape>" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NodeConstraintComponent ;
            sh:sourceShape _:naa093cd7dead42379b06b5799f03c34db1 ;
            sh:value <http://example.org/entities/Mary> ],
        [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Peter> ;
            sh:resultMessage "Value does not conform to Shape <http://example.org/entities/PeterKnowsShape>" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstrain

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/Peter>,:knows,<http://example.org/entities/Franz>,sh:NodeConstraintComponent,naa093cd7dead42379b06b5799f03c34db1,Value does not conform to Shape <http://exampl...
1,<http://example.org/entities/Peter>,:knows,<http://example.org/entities/Mary>,sh:NodeConstraintComponent,naa093cd7dead42379b06b5799f03c34db1,Value does not conform to Shape <http://exampl...
2,<http://example.org/entities/Peter>,:knows,<http://example.org/entities/Josef>,sh:NodeConstraintComponent,naa093cd7dead42379b06b5799f03c34db1,Value does not conform to Shape <http://exampl...


9. TASK

Das Nettogewicht eines Produkts darf nicht größer sein als das Bruttogewicht.
Verwenden Sie sh:lessThanOrEquals

In [22]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<Cookies>  rdf:type   <Product> ;
        :grossWeight  0.2 ;
        :netWeight    0.12 .

<Milk>  rdf:type      <Product> ;
        :grossWeight  1.1 ;
        :netWeight    1 .

<Peter>  rdf:type     <Person> ;
        :grossWeight  74 ;
        :netWeight    72 .

<Bread>  rdf:type     <Product> ;
        :grossWeight  1.1 ;
        :netWeight    1.2 .
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

[] a sh:NodeShape;
sh:targetClass <Product>;
sh:property[sh:path :netWeight; sh:lessThanOrEquals :grossWeight ].

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/properties/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Bread> ;
            sh:resultMessage "Value of <http://example.org/entities/Bread>->:grossWeight < Literal(\"1.2\", datatype=xsd:decimal)" ;
            sh:resultPath :netWeight ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:LessThanOrEqualsConstraintComponent ;
            sh:sourceShape [ sh:lessThanOrEquals :grossWeight ;
                    sh:path :netWeight ] ;
            sh:value 1.2 ] .




,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/Bread>,:netWeight,1.2,sh:LessThanOrEqualsConstraintComponent,ne40977a4c6344958a3476191a579f347b2,Value of <http://example.org/entities/Bread>->...


10. TASK

Die Objekte von knows-Statements haben einen Namen (Property :name) oder haben einen Vornamen (:givenName) und einen Nachnamen (:familyName). eine

In [23]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<Sara>  :givenName  "Sarah" .

<John>  :familyName  "Black" .

<Mary>  :knows  <Pete> .

<Bob>   :familyName  "Builder" ;
        :givenName   "Bob" .

<Pete>  :knows  <Bob> , <Sara> , <John> ;
        :name   "Peter Parker" .
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<KnowsShape> a sh:NodeShape;
sh:targetObjectsOf :knows;
sh:or ( 
[
sh:property[sh:path :name; sh:minCount 1] 
] 
[
sh:property[sh:path :givenName; sh:minCount 1];
sh:property[sh:path :familyName; sh:minCount 1] 
]
).

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Sara> ;
            sh:resultMessage "Node <http://example.org/entities/Sara> does not conform to one or more shapes in [ sh:property [ sh:minCount Literal(\"1\", datatype=xsd:integer) ; sh:path :name ] ] , [ sh:property [ sh:minCount Literal(\"1\", datatype=xsd:integer) ; sh:path :familyName ], [ sh:minCount Literal(\"1\", datatype=xsd:integer) ; sh:path :givenName ] ]" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:OrConstraintComponent ;
            sh:sourceShape <http://example.org/entities/KnowsShape> ;
            sh:value <http://example.org/entities/Sara> ],
        [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/John> ;
            sh:resultMessage "Node <http://ex

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/Sara>,None,<http://example.org/entities/Sara>,sh:OrConstraintComponent,<http://example.org/entities/KnowsShape>,Node <http://example.org/entities/Sara> does n...
1,<http://example.org/entities/John>,None,<http://example.org/entities/John>,sh:OrConstraintComponent,<http://example.org/entities/KnowsShape>,Node <http://example.org/entities/John> does n...
